## Bộ phân loại ẩm thực 2

Trong bài học phân loại thứ hai này, chúng ta sẽ khám phá `nhiều cách hơn` để phân loại dữ liệu dạng danh mục. Chúng ta cũng sẽ tìm hiểu về hậu quả của việc chọn một bộ phân loại thay vì bộ phân loại khác.

### [**Câu hỏi trước bài giảng**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/23/)

### **Điều kiện tiên quyết**

Chúng tôi giả định rằng bạn đã hoàn thành các bài học trước vì chúng ta sẽ tiếp tục sử dụng một số khái niệm đã học trước đó.

Đối với bài học này, chúng ta sẽ cần các gói sau:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) là một [bộ sưu tập các gói R](https://www.tidyverse.org/packages) được thiết kế để làm cho khoa học dữ liệu nhanh hơn, dễ dàng hơn và thú vị hơn!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) là một [khung làm việc](https://www.tidymodels.org/packages/) bao gồm các gói dành cho mô hình hóa và học máy.

-   `themis`: [gói themis](https://themis.tidymodels.org/) cung cấp các bước bổ sung trong công thức để xử lý dữ liệu không cân bằng.

Bạn có thể cài đặt chúng bằng lệnh:

`install.packages(c("tidyverse", "tidymodels", "kernlab", "themis", "ranger", "xgboost", "kknn"))`

Ngoài ra, đoạn mã dưới đây sẽ kiểm tra xem bạn đã có các gói cần thiết để hoàn thành module này chưa và cài đặt chúng cho bạn nếu chúng bị thiếu.


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, themis, kernlab, ranger, xgboost, kknn)

## **1. Một bản đồ phân loại**

Trong [bài học trước](https://github.com/microsoft/ML-For-Beginners/tree/main/4-Classification/2-Classifiers-1), chúng ta đã cố gắng giải quyết câu hỏi: làm thế nào để chọn giữa nhiều mô hình? Phần lớn điều này phụ thuộc vào đặc điểm của dữ liệu và loại vấn đề mà chúng ta muốn giải quyết (ví dụ như phân loại hay hồi quy?)

Trước đây, chúng ta đã tìm hiểu về các tùy chọn khác nhau khi phân loại dữ liệu bằng bảng gian lận của Microsoft. Framework Machine Learning của Python, Scikit-learn, cung cấp một bảng gian lận tương tự nhưng chi tiết hơn, giúp bạn thu hẹp các bộ ước lượng (một thuật ngữ khác cho các bộ phân loại):

<p >
   <img src="../../images/map.png"
   width="700"/>
   <figcaption></figcaption>


> Mẹo: [truy cập bản đồ trực tuyến này](https://scikit-learn.org/stable/tutorial/machine_learning_map/) và nhấp theo đường dẫn để đọc tài liệu.
>
> Trang [tham khảo Tidymodels](https://www.tidymodels.org/find/parsnip/#models) cũng cung cấp tài liệu tuyệt vời về các loại mô hình khác nhau.

### **Kế hoạch** 🗺️

Bản đồ này rất hữu ích khi bạn đã hiểu rõ dữ liệu của mình, vì bạn có thể 'đi bộ' theo các đường dẫn để đưa ra quyết định:

-   Chúng ta có \>50 mẫu

-   Chúng ta muốn dự đoán một danh mục

-   Chúng ta có dữ liệu được gắn nhãn

-   Chúng ta có ít hơn 100K mẫu

-   ✨ Chúng ta có thể chọn Linear SVC

-   Nếu cách đó không hiệu quả, vì chúng ta có dữ liệu dạng số

    -   Chúng ta có thể thử ✨ KNeighbors Classifier

        -   Nếu cách đó không hiệu quả, thử ✨ SVC và ✨ Ensemble Classifiers

Đây là một lộ trình rất hữu ích để làm theo. Bây giờ, hãy bắt đầu ngay với [tidymodels](https://www.tidymodels.org/): một bộ sưu tập các gói R nhất quán và linh hoạt được phát triển để khuyến khích thực hành thống kê tốt 😊.

## 2. Chia dữ liệu và xử lý tập dữ liệu không cân bằng.

Từ các bài học trước, chúng ta đã học rằng có một tập hợp các thành phần chung giữa các món ăn của chúng ta. Ngoài ra, có sự phân bố không đồng đều về số lượng món ăn.

Chúng ta sẽ xử lý những điều này bằng cách:

-   Loại bỏ các thành phần phổ biến nhất gây nhầm lẫn giữa các món ăn khác nhau, sử dụng `dplyr::select()`.

-   Sử dụng một `recipe` để tiền xử lý dữ liệu, chuẩn bị cho việc mô hình hóa bằng cách áp dụng thuật toán `over-sampling`.

Chúng ta đã xem qua những điều trên trong bài học trước nên việc này sẽ rất dễ dàng 🥳!


In [ ]:
# Load the core Tidyverse and Tidymodels packages
library(tidyverse)
library(tidymodels)

# Load the original cuisines data
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/4-Classification/data/cuisines.csv")

# Drop id column, rice, garlic and ginger from our original data set
df_select <- df %>% 
  select(-c(1, rice, garlic, ginger)) %>%
  # Encode cuisine column as categorical
  mutate(cuisine = factor(cuisine))


# Create data split specification
set.seed(2056)
cuisines_split <- initial_split(data = df_select,
                                strata = cuisine,
                                prop = 0.7)

# Extract the data in each split
cuisines_train <- training(cuisines_split)
cuisines_test <- testing(cuisines_split)

# Display distribution of cuisines in the training set
cuisines_train %>% 
  count(cuisine) %>% 
  arrange(desc(n))

### Xử lý dữ liệu không cân bằng

Dữ liệu không cân bằng thường ảnh hưởng tiêu cực đến hiệu suất của mô hình. Nhiều mô hình hoạt động tốt nhất khi số lượng quan sát là bằng nhau và do đó thường gặp khó khăn với dữ liệu không cân bằng.

Có hai cách chính để xử lý các tập dữ liệu không cân bằng:

-   thêm các quan sát vào lớp thiểu số: `Over-sampling`, ví dụ sử dụng thuật toán SMOTE, thuật toán này tạo ra các ví dụ mới cho lớp thiểu số một cách tổng hợp bằng cách sử dụng các điểm lân cận gần nhất của các trường hợp này.

-   loại bỏ các quan sát từ lớp đa số: `Under-sampling`

Trong bài học trước, chúng ta đã minh họa cách xử lý các tập dữ liệu không cân bằng bằng cách sử dụng một `recipe`. Recipe có thể được xem như một bản thiết kế mô tả các bước cần áp dụng cho một tập dữ liệu để chuẩn bị cho việc phân tích dữ liệu. Trong trường hợp của chúng ta, mục tiêu là có một phân phối đồng đều về số lượng các loại món ăn trong `training set`. Hãy bắt đầu ngay thôi.


In [ ]:
# Load themis package for dealing with imbalanced data
library(themis)

# Create a recipe for preprocessing training data
cuisines_recipe <- recipe(cuisine ~ ., data = cuisines_train) %>%
  step_smote(cuisine) 

# Print recipe
cuisines_recipe

Bây giờ chúng ta sẵn sàng để huấn luyện các mô hình 👩‍💻👨‍💻!

## 3. Vượt ra ngoài các mô hình hồi quy đa thức

Trong bài học trước, chúng ta đã tìm hiểu về các mô hình hồi quy đa thức. Hãy khám phá một số mô hình linh hoạt hơn cho bài toán phân loại.

### Máy Vector Hỗ Trợ (Support Vector Machines)

Trong bối cảnh phân loại, `Máy Vector Hỗ Trợ` là một kỹ thuật học máy cố gắng tìm một *siêu phẳng* (*hyperplane*) để "tách biệt tốt nhất" các lớp. Hãy xem một ví dụ đơn giản:

<p >
   <img src="../../images/svm.png"
   width="300"/>
   <figcaption>https://commons.wikimedia.org/w/index.php?curid=22877598</figcaption>


H1~ không tách biệt các lớp. H2~ có tách biệt, nhưng chỉ với một khoảng cách nhỏ. H3~ tách biệt chúng với khoảng cách tối đa.

#### Bộ phân loại tuyến tính SVM

Hỗ trợ phân cụm vector (SVC) là một nhánh của họ các kỹ thuật máy học (ML) thuộc máy vector hỗ trợ (SVM). Trong SVC, siêu phẳng được chọn để phân tách `phần lớn` các quan sát huấn luyện một cách chính xác, nhưng `có thể phân loại sai` một vài quan sát. Bằng cách cho phép một số điểm nằm ở phía sai, SVM trở nên mạnh mẽ hơn trước các giá trị ngoại lai, do đó cải thiện khả năng tổng quát hóa với dữ liệu mới. Tham số điều chỉnh sự vi phạm này được gọi là `cost`, với giá trị mặc định là 1 (xem `help("svm_poly")`).

Hãy tạo một SVC tuyến tính bằng cách đặt `degree = 1` trong một mô hình SVM đa thức.


In [ ]:
# Make a linear SVC specification
svc_linear_spec <- svm_poly(degree = 1) %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svc_linear_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svc_linear_spec)

# Print out workflow
svc_linear_wf

Bây giờ khi chúng ta đã lưu lại các bước tiền xử lý và đặc tả mô hình vào một *workflow*, chúng ta có thể tiến hành huấn luyện SVC tuyến tính và đánh giá kết quả trong quá trình đó. Để đo lường hiệu suất, hãy tạo một tập hợp các chỉ số để đánh giá: `accuracy`, `sensitivity`, `Positive Predicted Value` và `F Measure`.

> `augment()` sẽ thêm cột (hoặc các cột) cho dự đoán vào dữ liệu được cung cấp.


In [ ]:
# Train a linear SVC model
svc_linear_fit <- svc_linear_wf %>% 
  fit(data = cuisines_train)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)


# Make predictions and Evaluate model performance
svc_linear_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

#### Máy Vector Hỗ Trợ

Máy vector hỗ trợ (SVM) là một sự mở rộng của bộ phân loại vector hỗ trợ nhằm đáp ứng ranh giới phi tuyến giữa các lớp. Về cơ bản, SVM sử dụng *kernel trick* để mở rộng không gian đặc trưng, thích nghi với các mối quan hệ phi tuyến giữa các lớp. Một hàm kernel phổ biến và cực kỳ linh hoạt được SVM sử dụng là *Hàm cơ sở xuyên tâm.* Hãy cùng xem cách nó hoạt động trên dữ liệu của chúng ta.


In [ ]:
set.seed(2056)

# Make an RBF SVM specification
svm_rbf_spec <- svm_rbf() %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svm_rbf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svm_rbf_spec)


# Train an RBF model
svm_rbf_fit <- svm_rbf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
svm_rbf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Tuyệt vời hơn 🤩!

> ✅ Vui lòng xem:
>
> -   [*Support Vector Machines*](https://bradleyboehmke.github.io/HOML/svm.html), Hands-on Machine Learning with R
>
> -   [*Support Vector Machines*](https://www.statlearning.com/), An Introduction to Statistical Learning with Applications in R
>
> để đọc thêm.

### Bộ phân loại Hàng xóm Gần nhất

*K*-nearest neighbor (KNN) là một thuật toán trong đó mỗi quan sát được dự đoán dựa trên *sự tương đồng* của nó với các quan sát khác.

Hãy thử áp dụng nó vào dữ liệu của chúng ta.


In [ ]:
# Make a KNN specification
knn_spec <- nearest_neighbor() %>% 
  set_engine("kknn") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
knn_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(knn_spec)

# Train a boosted tree model
knn_wf_fit <- knn_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
knn_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Có vẻ như mô hình này không hoạt động tốt lắm. Có lẽ việc thay đổi các tham số của mô hình (xem `help("nearest_neighbor")`) sẽ cải thiện hiệu suất của mô hình. Hãy chắc chắn thử nghiệm điều này.

> ✅ Vui lòng tham khảo:
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> để tìm hiểu thêm về bộ phân loại *K*-Nearest Neighbors.

### Bộ phân loại tập hợp (Ensemble classifiers)

Các thuật toán tập hợp hoạt động bằng cách kết hợp nhiều bộ ước lượng cơ sở để tạo ra một mô hình tối ưu thông qua:

`bagging`: áp dụng một *hàm trung bình* cho một tập hợp các mô hình cơ sở

`boosting`: xây dựng một chuỗi các mô hình, trong đó mỗi mô hình kế tiếp cải thiện hiệu suất dự đoán dựa trên mô hình trước.

Hãy bắt đầu bằng cách thử nghiệm mô hình Random Forest, mô hình này xây dựng một tập hợp lớn các cây quyết định, sau đó áp dụng một hàm trung bình để tạo ra một mô hình tổng thể tốt hơn.


In [ ]:
# Make a random forest specification
rf_spec <- rand_forest() %>% 
  set_engine("ranger") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
rf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(rf_spec)

# Train a random forest model
rf_wf_fit <- rf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
rf_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Làm tốt lắm 👏!

Hãy cùng thử nghiệm với mô hình Boosted Tree.

Boosted Tree định nghĩa một phương pháp tập hợp, tạo ra một chuỗi các cây quyết định tuần tự, trong đó mỗi cây phụ thuộc vào kết quả của các cây trước đó nhằm giảm dần lỗi. Phương pháp này tập trung vào trọng số của các mục bị phân loại sai và điều chỉnh việc khớp cho bộ phân loại tiếp theo để sửa lỗi.

Có nhiều cách khác nhau để khớp mô hình này (xem `help("boost_tree")`). Trong ví dụ này, chúng ta sẽ khớp Boosted trees thông qua công cụ `xgboost`.


In [ ]:
# Make a boosted tree specification
boost_spec <- boost_tree(trees = 200) %>% 
  set_engine("xgboost") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
boost_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(boost_spec)

# Train a boosted tree model
boost_wf_fit <- boost_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
boost_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

> ✅ Vui lòng xem:
>
> -   [Machine Learning for Social Scientists](https://cimentadaj.github.io/ml_socsci/tree-based-methods.html#random-forests)
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> -   <https://algotech.netlify.app/blog/xgboost/> - Khám phá mô hình AdaBoost, một lựa chọn thay thế tốt cho xgboost.
>
> để tìm hiểu thêm về các bộ phân loại Ensemble.

## 4. Bổ sung - so sánh nhiều mô hình

Chúng ta đã xây dựng khá nhiều mô hình trong bài thực hành này 🙌. Việc tạo ra nhiều quy trình làm việc từ các bộ tiền xử lý và/hoặc các đặc tả mô hình khác nhau, sau đó tính toán từng chỉ số hiệu suất một cách riêng lẻ có thể trở nên tẻ nhạt hoặc khó khăn.

Hãy xem liệu chúng ta có thể giải quyết vấn đề này bằng cách tạo một hàm để huấn luyện danh sách các quy trình làm việc trên tập huấn luyện, sau đó trả về các chỉ số hiệu suất dựa trên tập kiểm tra. Chúng ta sẽ sử dụng `map()` và `map_dfr()` từ gói [purrr](https://purrr.tidyverse.org/) để áp dụng các hàm cho từng phần tử trong danh sách.

> Các hàm [`map()`](https://purrr.tidyverse.org/reference/map.html) cho phép bạn thay thế nhiều vòng lặp for bằng mã ngắn gọn hơn và dễ đọc hơn. Nơi tốt nhất để tìm hiểu về các hàm [`map()`](https://purrr.tidyverse.org/reference/map.html) là chương [iteration](http://r4ds.had.co.nz/iteration.html) trong sách R for Data Science.


In [ ]:
set.seed(2056)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)

# Define a function that returns performance metrics
compare_models <- function(workflow_list, train_set, test_set){
  
  suppressWarnings(
    # Fit each model to the train_set
    map(workflow_list, fit, data = train_set) %>% 
    # Make predictions on the test set
      map_dfr(augment, new_data = test_set, .id = "model") %>%
    # Select desired columns
      select(model, cuisine, .pred_class) %>% 
    # Evaluate model performance
      group_by(model) %>% 
      eval_metrics(truth = cuisine, estimate = .pred_class) %>% 
      ungroup()
  )
  
} # End of function

In [ ]:
# Make a list of workflows
workflow_list <- list(
  "svc" = svc_linear_wf,
  "svm" = svm_rbf_wf,
  "knn" = knn_wf,
  "random_forest" = rf_wf,
  "xgboost" = boost_wf)

# Call the function
set.seed(2056)
perf_metrics <- compare_models(workflow_list = workflow_list, train_set = cuisines_train, test_set = cuisines_test)

# Print out performance metrics
perf_metrics %>% 
  group_by(.metric) %>% 
  arrange(desc(.estimate)) %>% 
  slice_head(n=7)

# Compare accuracy
perf_metrics %>% 
  filter(.metric == "accuracy") %>% 
  arrange(desc(.estimate))


[**workflowset**](https://workflowsets.tidymodels.org/) cho phép người dùng tạo và dễ dàng huấn luyện một số lượng lớn mô hình, nhưng chủ yếu được thiết kế để hoạt động với các kỹ thuật lấy mẫu lại như `cross-validation`, một phương pháp mà chúng ta chưa đề cập đến.

## **🚀Thử thách**

Mỗi kỹ thuật này có một số lượng lớn các tham số mà bạn có thể điều chỉnh, ví dụ như `cost` trong SVMs, `neighbors` trong KNN, `mtry` (Các Dự đoán Được Chọn Ngẫu Nhiên) trong Random Forest.

Hãy nghiên cứu các tham số mặc định của từng kỹ thuật và suy nghĩ về việc điều chỉnh các tham số này sẽ ảnh hưởng như thế nào đến chất lượng của mô hình.

Để tìm hiểu thêm về một mô hình cụ thể và các tham số của nó, sử dụng: `help("model")` ví dụ `help("rand_forest")`

> Trong thực tế, chúng ta thường *ước lượng* các *giá trị tốt nhất* cho các tham số này bằng cách huấn luyện nhiều mô hình trên một `bộ dữ liệu mô phỏng` và đo lường hiệu suất của tất cả các mô hình này. Quá trình này được gọi là **tuning**.

### [**Câu hỏi sau bài giảng**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/24/)

### **Ôn tập & Tự học**

Có rất nhiều thuật ngữ chuyên ngành trong các bài học này, vì vậy hãy dành một chút thời gian để xem lại [danh sách này](https://docs.microsoft.com/dotnet/machine-learning/resources/glossary?WT.mc_id=academic-77952-leestott) các thuật ngữ hữu ích!

#### CẢM ƠN ĐẾN:

[`Allison Horst`](https://twitter.com/allison_horst/) vì đã tạo ra những hình minh họa tuyệt vời giúp R trở nên thân thiện và hấp dẫn hơn. Tìm thêm các hình minh họa tại [bộ sưu tập của cô ấy](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

[Cassie Breviu](https://www.twitter.com/cassieview) và [Jen Looper](https://www.twitter.com/jenlooper) vì đã tạo ra phiên bản Python gốc của module này ♥️

Chúc học tập vui vẻ,

[Eric](https://twitter.com/ericntay), Đại sứ Sinh viên Microsoft Learn Vàng.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="569"/>
   <figcaption>Tác phẩm nghệ thuật của @allison_horst</figcaption>



---

**Tuyên bố miễn trừ trách nhiệm**:  
Tài liệu này đã được dịch bằng dịch vụ dịch thuật AI [Co-op Translator](https://github.com/Azure/co-op-translator). Mặc dù chúng tôi cố gắng đảm bảo độ chính xác, xin lưu ý rằng các bản dịch tự động có thể chứa lỗi hoặc không chính xác. Tài liệu gốc bằng ngôn ngữ bản địa nên được coi là nguồn thông tin chính thức. Đối với các thông tin quan trọng, khuyến nghị sử dụng dịch vụ dịch thuật chuyên nghiệp bởi con người. Chúng tôi không chịu trách nhiệm cho bất kỳ sự hiểu lầm hoặc diễn giải sai nào phát sinh từ việc sử dụng bản dịch này.
